<a href="https://colab.research.google.com/github/JJoshK/-JJoshK-.github.io/blob/master/Flux_Balance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cobra

# 모델 불러오기 (파일명이 iECD_1391.json 인지 확인하세요)
model = cobra.io.load_json_model("iECD_1391.json")

# 모델 기본 정보 출력
print(f"모델 이름: {model.id}")
print(f"반응(Reactions) 수: {len(model.reactions)}")
print(f"대사산물(Metabolites) 수: {len(model.metabolites)}")

# 기본 상태에서 시뮬레이션 실행 (보통 세포 성장 최대화가 기본 설정입니다)
solution = model.optimize()
print(f"최적 성장 속도: {solution.objective_value}")

모델 이름: iECD_1391
반응(Reactions) 수: 2741
대사산물(Metabolites) 수: 1943
최적 성장 속도: 0.9756145115851301


In [5]:
# 모델에서 반응 삭제
model.remove_reactions([
    model.reactions.get_by_id('EX_glc__e'),
    model.reactions.get_by_id('HK1'),
    model.reactions.get_by_id('ATP_synthase')
])

print(f"반응 삭제 후 모델의 반응 수: {len(model.reactions)}")

# 삭제 확인 (에러가 발생해야 정상)
try:
    model.reactions.get_by_id('EX_glc__e')
except KeyError as e:
    print(f"EX_glc__e 반응이 삭제되었습니다: {e}")

KeyError: 'EX_glc__e'

위 코드를 실행하면 이전에 추가했던 `EX_glc__e`, `HK1`, `ATP_synthase` 반응들이 모델에서 삭제됩니다. 삭제 후 `model.reactions.get_by_id()`로 해당 반응을 찾으려고 하면 `KeyError`가 발생하여 성공적으로 삭제되었음을 확인할 수 있습니다.

In [3]:
import cobra
from cobra import Metabolite, Reaction

# 새로운 대사산물 정의
glucose = Metabolite('glucose_e', name='Glucose external', compartment='e')
atp = Metabolite('atp_c', name='ATP cytoplasmic', compartment='c')
adp = Metabolite('adp_c', name='ADP cytoplasmic', compartment='c')
pi = Metabolite('pi_c', name='Phosphate cytoplasmic', compartment='c')
glucose_6p = Metabolite('g6p_c', name='Glucose-6-phosphate cytoplasmic', compartment='c')

# 새로운 반응 정의

# 1. 포도당 외부 유입 반응 (Exchange Reaction)
glucose_exchange_reaction = Reaction('EX_glc__e')
glucose_exchange_reaction.name = 'Glucose exchange'
glucose_exchange_reaction.lower_bound = -10.0  # 외부에서 10 단위까지 유입 가능
glucose_exchange_reaction.upper_bound = 1000.0 # 내부에서 외부로 유출 가능 (거의 무제한)
glucose_exchange_reaction.add_metabolites({
glucose: -1.0 # 외부에서 내부로 유입
})

# 2. 해당 과정 첫 번째 단계 (Glycolysis - Hexokinase)
glycolysis_reaction = Reaction('HK1')
glycolysis_reaction.name = 'Hexokinase'
glycolysis_reaction.lower_bound = 0.0 # 정방향만 가능
glycolysis_reaction.upper_bound = 1000.0
glycolysis_reaction.add_metabolites({
glucose: -1.0,
atp: -1.0,
glucose_6p: 1.0,
adp: 1.0
})

# 3. ATP 합성 효소 (예시 반응, 실제 모델에 있는 ATP synthase와 다를 수 있음)
atp_synthase_reaction = Reaction('ATP_synthase')
atp_synthase_reaction.name = 'ATP Synthase'
atp_synthase_reaction.lower_bound = -1000.0 # 양방향 가능
atp_synthase_reaction.upper_bound = 1000.0
atp_synthase_reaction.add_metabolites({
adp: -1.0,
pi: -1.0,
atp: 1.0
})


# 기존 모델에 반응 추가
model.add_reactions([
glucose_exchange_reaction,
glycolysis_reaction,
atp_synthase_reaction
])

print(f"모델에 추가된 반응 수: {len(model.reactions)}")

# 추가된 반응 확인
print("\n새롭게 추가된 반응:")
print(model.reactions.get_by_id('EX_glc__e'))
print(model.reactions.get_by_id('HK1'))
print(model.reactions.get_by_id('ATP_synthase'))

# 새로운 대사산물 확인 (자동으로 추가됨)
print("\n새롭게 추가된 대사산물:")
print(model.metabolites.get_by_id('glucose_e'))
print(model.metabolites.get_by_id('g6p_c'))


모델에 추가된 반응 수: 2744

새롭게 추가된 반응:
EX_glc__e: glucose_e <=> 
HK1: atp_c + glucose_e --> adp_c + g6p_c
ATP_synthase: adp_c + pi_c <=> atp_c

새롭게 추가된 대사산물:
glucose_e
g6p_c


위 코드는 `cobra` 라이브러리를 사용하여 새로운 대사산물(예: `glucose_e`, `atp_c`)과 반응(예: `EX_glc__e`, `HK1`)을 정의하고, 이를 기존 `model` 객체에 추가합니다.

각 반응에 대해 다음과 같은 속성을 설정할 수 있습니다:
- `id`: 반응의 고유 ID
- `name`: 반응의 설명적인 이름
- `lower_bound`, `upper_bound`: 반응 플럭스의 최솟값과 최댓값. 이는 반응이 단방향인지 양방향인지, 그리고 얼마나 많은 양이 이동할 수 있는지를 결정합니다.
- `add_metabolites`: 반응에 참여하는 대사산물과 그 계수(stoichiometric coefficient)를 정의합니다. 계수가 음수이면 반응물, 양수이면 생성물입니다.

모델에 반응을 추가한 후에는 `model.reactions.get_by_id()`를 사용하여 추가된 반응을 확인할 수 있습니다.

In [9]:
# 'idi'가 포함된 모든 반응의 ID와 이름을 출력
for rxn in model.reactions:
    if 'idi' in rxn.id.lower():
        print(f"ID: {rxn.id} | Name: {rxn.name}")

In [11]:
# IPP(ipdp_c)가 관여하는 모든 반응 찾기
met = model.metabolites.get_by_id('ipdp_c')
for rxn in met.reactions:
    print(f"ID: {rxn.id} | Equation: {rxn.reaction}")

ID: DMATT | Equation: dmpp_c + ipdp_c --> grdp_c + ppi_c
ID: GRTT | Equation: grdp_c + ipdp_c --> frdp_c + ppi_c
ID: IPDDI | Equation: ipdp_c <=> dmpp_c
ID: OCTDPS | Equation: frdp_c + 5.0 ipdp_c --> octdp_c + 5.0 ppi_c
ID: IPDPS | Equation: h2mb4p_c + h_c + nadh_c --> h2o_c + ipdp_c + nad_c
ID: UDCPDPS | Equation: frdp_c + 8.0 ipdp_c --> 8.0 ppi_c + udcpdp_c


In [13]:
# HMG-CoA(hmgcoa_c)가 관여하는 모든 반응을 찾아 기존 ID 확인
try:
    met = model.metabolites.get_by_id('hmgcoa_c')
    for rxn in met.reactions:
        print(f"이미 존재하는 반응 ID: {rxn.id} | 식: {rxn.reaction}")
except:
    print("HMG-CoA 대사산물이 모델에 없습니다.")

HMG-CoA 대사산물이 모델에 없습니다.


In [16]:
# 확인하고 싶은 대사산물 ID 리스트
target_metabolites = ['aaccoa_c', 'hmgcoa_c', 'mev_c', 'mev__R_c', 'mev5p_c', 'mev5pp_c', 'ipdp_c', 'dmapp_c', 'dmpp_c']

print("--- 대사산물 존재 여부 확인 결과 ---")
for met_id in target_metabolites:
    if met_id in model.metabolites:
        met = model.metabolites.get_by_id(met_id)
        print(f"[존재] {met_id} : {met.name}")
    else:
        print(f"[없음] {met_id}")

--- 대사산물 존재 여부 확인 결과 ---
[없음] aaccoa_c
[없음] hmgcoa_c
[없음] mev_c
[없음] mev__R_c
[없음] mev5p_c
[없음] mev5pp_c
[존재] ipdp_c : Isopentenyl diphosphate
[없음] dmapp_c
[존재] dmpp_c : Dimethylallyl diphosphate


In [21]:
import cobra
from cobra import Reaction, Metabolite

# 1. 모델 로드 (업로드한 파일명이 다를 경우 수정하세요)
model = cobra.io.load_json_model("iECD_1391.json")

# 2. 모델에 없는 MVA 경로 중간 대사산물 정의 및 추가
# (ipdp_c와 dmpp_c는 이미 존재하므로 제외하고 새로 정의합니다)
new_metabolites = [
    Metabolite('aaccoa_c', formula='C25H40N7O18P3S', name='Acetoacetyl-CoA', compartment='c'),
    Metabolite('hmgcoa_c', formula='C27H44N7O20P3S', name='Hydroxymethylglutaryl-CoA', compartment='c'),
    Metabolite('mev_c', formula='C6H12O4', name='Mevalonate', compartment='c'),
    Metabolite('mev5p_c', formula='C6H13O7P', name='Mevalonate-5-phosphate', compartment='c'),
    Metabolite('mev5pp_c', formula='C6H14O10P2', name='Mevalonate-5-pyrophosphate', compartment='c'),
    Metabolite('isoprene_e', formula='C5H8', name='Isoprene', compartment='e') # 'c'를 'e'로 변경
]
model.add_metabolites(new_metabolites)

# 3. 7가지 효소 반응 추가
reactions = []

# MvaE (Thiolase): 2 accoa -> aaccoa + coa
r1 = Reaction('MvaE_T')
r1.add_metabolites({model.metabolites.accoa_c: -2, model.metabolites.aaccoa_c: 1, model.metabolites.coa_c: 1})
reactions.append(r1)

# MvaS (HMG-CoA Synthase): accoa + aaccoa + h2o -> hmgcoa + coa + h
r2 = Reaction('MvaS')
r2.add_metabolites({
    model.metabolites.accoa_c: -1, model.metabolites.aaccoa_c: -1, model.metabolites.h2o_c: -1,
    model.metabolites.hmgcoa_c: 1, model.metabolites.coa_c: 1, model.metabolites.h_c: 1
})
reactions.append(r2)

# MvaE (Reductase): hmgcoa + 2 nadph + 2 h -> mev + coa + 2 nadp
r3 = Reaction('MvaE_R')
r3.add_metabolites({
    model.metabolites.hmgcoa_c: -1, model.metabolites.nadph_c: -2, model.metabolites.h_c: -2,
    model.metabolites.mev_c: 1, model.metabolites.coa_c: 1, model.metabolites.nadp_c: 2
})
reactions.append(r3)

# MK (Mevalonate Kinase): mev + atp -> mev5p + adp + h
r4 = Reaction('MK')
r4.add_metabolites({model.metabolites.mev_c: -1, model.metabolites.atp_c: -1, model.metabolites.mev5p_c: 1, model.metabolites.adp_c: 1, model.metabolites.h_c: 1})
reactions.append(r4)

# PMK (Phosphomevalonate Kinase): mev5p + atp -> mev5pp + adp
r5 = Reaction('PMK')
r5.add_metabolites({model.metabolites.mev5p_c: -1, model.metabolites.atp_c: -1, model.metabolites.mev5pp_c: 1, model.metabolites.adp_c: 1})
reactions.append(r5)

# PMD (Mevalonate pyrophosphate decarboxylase): mev5pp + atp -> ipdp_c + adp + pi + co2
r6 = Reaction('PMD')
r6.add_metabolites({
    model.metabolites.mev5pp_c: -1, model.metabolites.atp_c: -1,
    model.metabolites.ipdp_c: 1, model.metabolites.adp_c: 1, model.metabolites.pi_c: 1, model.metabolites.co2_c: 1
})
reactions.append(r6)

# ISPS (Isoprene Synthase): dmpp_c -> isoprene_e + ppi (기존 dmpp_c ID 사용) # isoprene_c를 isoprene_e로 변경
r7 = Reaction('ISPS')
r7.add_metabolites({model.metabolites.dmpp_c: -1, model.metabolites.isoprene_e: 1, model.metabolites.ppi_c: 1}) # isoprene_c를 isoprene_e로 변경
reactions.append(r7)

model.add_reactions(reactions)

# 4. 기존 IDI(IPDDI) 반응 가역성 확보 (이미 되어있지만 확인 차원)
model.reactions.IPDDI.lower_bound = -1000.0

# 5. 시뮬레이션 설정: 이소프렌 배출 반응 추가 및 목적함수 설정
# 'id' 인수는 add_boundary에 지원되지 않으므로 제거합니다.
model.add_boundary(model.metabolites.isoprene_e, type='exchange') # isoprene_c를 isoprene_e로 변경
model.objective = 'ISPS'

# 6. 결과 출력
solution = model.optimize()
print(f"--- 시뮬레이션 결과 ---")
if solution.status == 'optimal':
    print(f"최대 이소프렌 생산 수율: {solution.objective_value:.4f} mmol/gDCW/hr")
else:
    print("최적해를 찾지 못했습니다. 경로 연결을 확인하세요.")

--- 시뮬레이션 결과 ---
최대 이소프렌 생산 수율: 8.1983 mmol/gDCW/hr
